In [ ]:
import logging
import os
import pandas as pd
import re

from pathlib import Path

# Disable logging of the program in the notebook
os.environ["LOGLEVEL"] = "CRITICAL"

logging.basicConfig()
logging.root.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('__main__')
logger.setLevel(os.environ.get("LOGLEVEL", logging.INFO))

pybooklogger = logging.getLogger('pybook')
pybooklogger.setLevel(logging.DEBUG)

import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.path.abspath("__file__")), '..')))
from experiments.setup import *
from experiments.tad_help import TadRunner

# Track Layout
Calculate the layout of the dutch railway system


In [ ]:
layout_file =   "../data/prorail/parsed/netherlands-schiphol.json"
layout = Layout(layout_file)

In [ ]:
scenario_files = Path("../data/prorail/scenarios/TAD/DenHelder")
save_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", "tad")
Path(save_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
path_data = []
time_data = []
comp_data = []

for scenario_file in os.listdir(scenario_files):
# for scenario_file in ["2025-07-21_3.json"]:
    tad_exp = TadRunner(layout, scenario_files / scenario_file, save_dir)
    experiments = tad_exp.run("3000", "e", "SGN", "HDR", default_direction=1)
    experiments[0].metadata = {'color': 'Red',   'label': '@SIPP',    'offset': 0, 'linestyle': 3}
    experiments[1].metadata = {'color': 'Blue',  'label': 'FlexSIPP', 'offset': 0, 'linestyle': 3}

    tad_exp.plot(experiments, save="3000e-Hdr")
    tad_exp.plot([experiments[0]], save="3000e-Hdr/atsipp")
    tad_exp.plot([experiments[1]], save="3000e-Hdr/flexsipp")

    path_data.extend(get_path_data(experiments, tad_exp.agent_df, scenario=scenario_file))
    time_data.extend([exp.get_running_time() | exp.get_label() | {'scenario': scenario_file} for exp in experiments])
    comp_data.extend([exp.get_complexity()   | exp.get_label() | {'scenario': scenario_file} for exp in experiments])

path_df = pd.DataFrame(path_data)
path_df.to_csv(save_dir + "/3000e-Hdr.csv")

time_df = pd.DataFrame(time_data)
time_df.to_csv(save_dir + "/3000e-Hdr-time.csv")

comp_df = pd.DataFrame(comp_data)
comp_df.to_csv(save_dir + "/3000e-Hdr-comp.csv")

MEtadata {'Nodes generated': '<872.735,1066.86,1338.09,1532.21>\\n<1066.86,inf,inf,inf>', 'Nodes decreased': '<1558.56,2160.62,2023.92,2625.97>\\n<1066.86,1558.56,2023.92,2023.92>', 'Nodes expanded': 'generated:'}


In [ ]:
save_dir = r"C:\Users\erick\Documents\uni\Thesis\delay-replannning\experiments\results\tad"
path_df = pd.read_csv(save_dir + "/3000e-Hdr.csv", index_col=0)
path_df

In [ ]:
allowed_delay = 180

def td_str(td):
    return ':'.join(re.split(r'[:.]+', str(td)) [1:3])

def extract_tipping_point(df):
    def apply_func(df):

        result=df.groupby("Delay Location").agg({
            "beta": "max",
            "Delay Amount": "max",
        })
        # result = result.loc[result['beta'] < 900]
        result["Tipping Point"] = result["beta"].apply(lambda x: td_str(timedelta(seconds=x)))

        def tp_finder(x):
            new_tp = x['beta'] - max(0, x['Delay Amount'] - allowed_delay)
            if new_tp > 0:
                return td_str(timedelta(seconds=new_tp))
            return "-"

        result[f"Tipping Point ({allowed_delay}s)"] = result.apply(tp_finder, axis=1)
        result["Delay Amount"] = result["Delay Amount"].apply(lambda x: td_str(timedelta(seconds=x)))
        return result.sort_values("Tipping Point", ascending=True).drop(columns=["beta"])

    df["Delay Location"] = df["Delay Location"].str.split("|").apply(lambda x: x[0])
    df = df.groupby(by='Train').apply(apply_func, include_groups=False)
    return df

tp_df = path_df.rename(columns={
    "delay_amount": "Delay Amount",
    "delay_location": "Delay Location",
    "trainNumber": "Train",
    "scenario": "Scenario",
    "label": "Label",
})
tp_df["Scenario"] = tp_df["Scenario"].apply(lambda x: x.split(".")[0])
tp_df["Delay Location"] = tp_df["Delay Location"].apply(lambda x: x.split("-")[1])
tp_df = tp_df.groupby(["Scenario"]).apply(extract_tipping_point, include_groups=False)
tp_df.loc["2025-07-21_3"].iloc[:, [1, 0]].to_latex(save_dir + "/3000e-Hdr.tex", escape=True)
tp_df.iloc[:, [1, 0, 2]]

# Experiment Runtime
Take a route of an agent with many stops, and run from start to every stop as an experiment

## Results


### ATF Plot

### Blocking staircase diagram
Showing the route of the agent with the most stops, its quite long.


In [ ]:
tad_exp = TadRunner(layout, scenario_files / "2025-07-21_3.json", save_dir)
experiments = tad_exp.run("3000", "e", "SGN", "HDR", default_direction=1, timeout=1)

In [ ]:
for exp in experiments:
    exp.s.plot(exp.agent.id, exp.block_intervals, exp.buffer_times, exp.recovery_times, True, start="SGN|", end="HDR", min_y=28.5*60, max_y=(28.5+60)*60, savefig=f"{save_dir}/2025-07-21_3/blocking_staircase_{exp.metadata['label'].replace(' ', '_')}.png")

### Time statistics

In [ ]:
def sum_cols(df1, cols, name):
    df2 = df1.drop(columns=cols)
    df2[name] = df1[cols].sum(axis=1)
    return df2

time_df = pd.DataFrame([exp.get_running_time() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

setup_cols = ["track graph creation", "routing graph creation"]
recompute_cols = ["unsafe interval generation", "safe interval generation", "bt and crt generation", "converting routes to blocks"]
search_cols = ["FlexSIPP search time"]

time_df = sum_cols(time_df, setup_cols, "Setup Time")
time_df = sum_cols(time_df, recompute_cols, "Recompute Time")
time_df = sum_cols(time_df, search_cols, "Search Time")
time_df

### Search Node Statistics


In [ ]:
nodes_df = pd.DataFrame([exp.get_complexity() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

nodes_df

### Output paths found


In [ ]:
for key, value in experiments[2].results[3].items():
    delayed_trains = {i: v for i,v in enumerate(value[0][4]) if float(v[0]) > 0}
    print(f"{key.replace('r-', '')}\nearliest departure: {int(min(float(value[0][1]), float(value[0][2])) / 60)}\ndepart before: {int(float(value[0][2]) / 60)}\narrive at: {int((float(value[0][1]) + float(value[0][3])) / 60)}\ndelays trains: {delayed_trains}")

### Path statistics


In [ ]:
for exp in experiments:
    print(f"Differend paths found for {exp.metadata['label']}: {sum(exp.results[2].values())}")